# authentication flow

video guide: [recording link](https://drive.google.com/file/d/1i_GdL8-ZlhQw9eyAHoNDJ8Z0o2GEGSyX/view?usp=sharing)

1. login device flow
2. uri complete to authorize
3. get client_token
4. insert client_token to with renew TTL to redis


- device flow guide: https://gitlab-master.nvidia.com/kaizen/auth/starfleet/sf-docs/-/blob/main/apis/user-authentication/device-authorization-endpoint.md
- refresh token guide: https://gitlab-master.nvidia.com/kaizen/auth/starfleet/sf-docs/-/blob/main/overview/extras/refresh-tokens.md?ref_type=heads

- token guide: https://gitlab-master.nvidia.com/kaizen/auth/starfleet/sf-docs/-/blob/main/apis/user-authentication/token-endpoint.md#poll-the-status-of-an-off-device-login

service admin:
- stage: https://stg.a12n.nvidia.com/services
- prod: https://a12n.nvidia.com/


In [ ]:
import os

from openai.resources.realtime import client_secrets

# Need input
CLIENT_IDS = {
    "nonprd": "LRddW8daXcXkHW_2nvQBJt1V969xHj6m6EHf-1S4eJU",
    "prd": "QHWXxZpiDCFZ8T9zCbX9QJUOotq8c6HGVlyYIvDGxXk"
}

CLIENT_SECRETS = {
    "nonprd": "sfcs-jknAAm9gXeD8Douhe1Wr3M61JMSuAN3l",
    "prd": "sfcs-DDDzquDqbbpe1F1kVGEWm7E6azzy4ZM0"
}

ENV_TO_RETRIEVE_TOKEN = "nonprd"

# user_id will be user for individual person, or could user unique bot id (in future when bot configuration is ready).
userid = ""

authorize_url_stg = 'https://stg.login.nvidia.com/device/authorize'
authorize_url_prd = 'https://login.nvidia.com/device/authorize'

token_url_stg = "https://stg.login.nvidia.com/token"
token_url_prd = "https://login.nvidia.com/token"

client_token_url_stg = "https://stg.login.nvidia.com/client_token"
client_token_url_prd = "https://login.nvidia.com/client_token"

In [ ]:
# Need input
# option: "stg", "prd"
print(f"✋Verify that we will retrieve for *{ENV_TO_RETRIEVE_TOKEN}* env")
# intentional env overwrite
env = "prd" if ENV_TO_RETRIEVE_TOKEN.lower() == "prd" else "nonprd"

In [ ]:
import requests, json

client_id = CLIENT_IDS.get(env)
client_secret = CLIENT_SECRETS.get(env)
assert client_id is not None

authorize_url = authorize_url_stg if env.lower() == "nonprd" else authorize_url_prd
token_url = token_url_stg if env.lower() == "nonprd" else token_url_prd
client_token_url = client_token_url_stg if env.lower() == "nonprd" else client_token_url_prd
values_dict = {"client_id": client_id, "client_secret": client_secret}

In [ ]:
device_id = 'thisisadeviceidfornvbotevaluation'
display_name = f"{userid or ''}DeviceFlow"
scope = 'openid profile email offline_access'

### Step 1: get Device Code and Authorize



In [ ]:
print("-> authorize_url: ", authorize_url)
print("-> client_id: ", client_id)

data = {
    'client_id': client_id,
    'device_id': device_id,
    'display_name': display_name,
    'scope': scope
}
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

response = requests.post(authorize_url, headers=headers, data=data)

response_dict = None
if response.status_code == 200:
    response_dict = json.loads(response.text)
    # print("And Type this code", response_dict["user_code"])

else:
    print(f"Invalid request, status code = {response.status_code}, {response.text}")

if response_dict:
    device_code = response_dict["device_code"]
    # print ("device_code: \n", device_code)

print("\nGo to the url endpoint in next code execution block for UI login \n")
print("1. click `Submit` ")
print("2. click `Continue`")
print("3. should see `You are logged in`")

In [ ]:
import webbrowser

webbrowser.open_new_tab(response_dict["verification_uri_complete"])
print("✋Verify complete this steps and able to see `You are logged in`")

###  Step 2: get Access Token

In [ ]:
# URL
print("-> token_url: ", token_url)
print("-> device_code: ", device_code)
data = {
    'device_code': device_code,
    'grant_type': 'urn:ietf:params:oauth:grant-type:device_code',
    'client_id': client_id,
}

token_response = requests.post(token_url, headers=headers, data=data)
refresh_token = None
access_token = None
if token_response.status_code == 200:
    token_response_dict = token_response.json()
    # print("Id token: \n", token_response_dict["id_token"])
    # values_dict["id_token"] = token_response_dict["id_token"]
    access_token = token_response_dict["access_token"]
    values_dict["access_token"] = access_token
    refresh_token = token_response_dict["refresh_token"]
    values_dict["refresh_token"] = refresh_token
    print("Access token: \n", token_response_dict["access_token"])
    print("Refresh token: \n", token_response_dict["refresh_token"])
    print("Expire:\n", token_response_dict["expires_in"])
else:
    print(f"Invalid request, status code = {response_dict.status_code}")


In [ ]:
# token_response_dict

In [ ]:
# short
refresh_token_data = {
    "grant_type": "refresh_token",
    "refresh_token": refresh_token,
}

# long?
refresh_token_data = {
    'client_id': client_id,
    'client_secret': client_secret,
    "grant_type": "refresh_token",
    "refresh_token": refresh_token,
}


refresh_token_response = requests.post(token_url, headers=headers, data=refresh_token_data)
if refresh_token_response.status_code == 200:
    refresh_token_response_dict = refresh_token_response.json()
    print("Id token: \n", refresh_token_response_dict["id_token"])
    values_dict["id_token"] = refresh_token_response_dict["id_token"]
    print("Expires in:\n", refresh_token_response_dict["expires_in"])
    values_dict["expires_in"] = refresh_token_response_dict["expires_in"]
    refresh_token = refresh_token_response_dict["refresh_token"]
    values_dict["refresh_token"] = refresh_token_response_dict["refresh_token"]
else:
    print(f"Invalid request, status code = {refresh_token_response.status_code}, {refresh_token_response.content}")

In [ ]:
refresh_token = refresh_token_response_dict["refresh_token"]
refresh_token, refresh_token_response_dict["expires_in"]

In [ ]:
# check again


# # long?
refresh_token_data = {
    'client_id': client_id,
    'client_secret': client_secret,
    "grant_type": "refresh_token",
    "refresh_token": refresh_token,
}

refresh_token_response = requests.post(token_url, headers=headers, data=refresh_token_data)
if refresh_token_response.status_code == 200:
    refresh_token_response_dict = refresh_token_response.json()
    print("Id token: \n", refresh_token_response_dict["id_token"])
    values_dict["id_token"] = refresh_token_response_dict["id_token"]
    print("Expires in:\n", refresh_token_response_dict["expires_in"])
    values_dict["expires_in"] = refresh_token_response_dict["expires_in"]
    refresh_token = refresh_token_response_dict["refresh_token"]
    values_dict["refresh_token"] = refresh_token_response_dict["refresh_token"]
else:
    print(f"Invalid request, status code = {refresh_token_response.status_code}")


## Prod token retrieval

In [ ]:
env = "prd"

In [ ]:
import requests, json

client_id = CLIENT_IDS.get(env)
client_secret = CLIENT_SECRETS.get(env)
assert client_id is not None

authorize_url = authorize_url_stg if env.lower() == "nonprd" else authorize_url_prd
token_url = token_url_stg if env.lower() == "nonprd" else token_url_prd
client_token_url = client_token_url_stg if env.lower() == "nonprd" else client_token_url_prd
values_dict_prd = {"client_id": client_id, "client_secret": client_secret}

### Step 1: Get device code

In [ ]:
print("-> authorize_url: ", authorize_url)
print("-> client_id: ", client_id)

data = {
    'client_id': client_id,
    'device_id': device_id,
    'display_name': display_name,
    'scope': scope
}
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

response = requests.post(authorize_url, headers=headers, data=data)

response_dict = None
if response.status_code == 200:
    response_dict = json.loads(response.text)
    # print("And Type this code", response_dict["user_code"])

else:
    print(f"Invalid request, status code = {response.status_code}, {response.text}")

if response_dict:
    device_code = response_dict["device_code"]
    # print ("device_code: \n", device_code)

print("\nGo to the url endpoint in next code execution block for UI login \n")
print("1. click `Submit` ")
print("2. click `Continue`")
print("3. should see `You are logged in`")

In [ ]:
import webbrowser

webbrowser.open_new_tab(response_dict["verification_uri_complete"])
print("✋Verify complete this steps and able to see `You are logged in`")

## Step 2: get authorized by refresh token

In [ ]:
# URL
print("-> token_url: ", token_url)
print("-> device_code: ", device_code)
data = {
    'device_code': device_code,
    'grant_type': 'urn:ietf:params:oauth:grant-type:device_code',
    'client_id': client_id,
}

token_response_prd = requests.post(token_url, headers=headers, data=data)
refresh_token = None
access_token = None
if token_response_prd.status_code == 200:
    token_response_prd_dict = token_response_prd.json()
    access_token = token_response_prd_dict["access_token"]
    values_dict_prd["access_token"] = access_token
    refresh_token = token_response_prd_dict["refresh_token"]
    values_dict_prd["refresh_token"] = refresh_token
    print("Access token: \n", token_response_prd_dict["access_token"])
    print("Refresh token: \n", token_response_prd_dict["refresh_token"])
    print("Expire:\n", token_response_prd_dict["expires_in"])
else:
    print(f"Invalid request, status code = {response_dict.status_code}")


In [ ]:
# short
refresh_token_data = {
    "grant_type": "refresh_token",
    "refresh_token": refresh_token,
}

# long?
refresh_token_data = {
    'client_id': client_id,
    'client_secret': client_secret,
    "grant_type": "refresh_token",
    "refresh_token": refresh_token,
}

refresh_token_prd_response = requests.post(token_url, headers=headers, data=refresh_token_data)
if refresh_token_prd_response.status_code == 200:
    refresh_token_response_prd_dict = refresh_token_prd_response.json()
    print("Id token: \n", refresh_token_response_prd_dict["id_token"])
    values_dict_prd["id_token"] = refresh_token_response_prd_dict["id_token"]
    print("Expires in:\n", refresh_token_response_prd_dict["expires_in"])
    values_dict_prd["expires_in"] = refresh_token_response_prd_dict["expires_in"]
    refresh_token = refresh_token_response_prd_dict["refresh_token"]
    values_dict_prd["refresh_token"] = refresh_token_response_prd_dict["refresh_token"]
else:
    print(f"Invalid request, status code = {refresh_token_prd_response.status_code}")


In [ ]:
refresh_token_prd = refresh_token_response_prd_dict["refresh_token"]
f"refresh token: {refresh_token_prd}", f"access token expired in: {refresh_token_response_dict['expires_in']}"

In [ ]:
print("✅  Use following command to update Redis cli")
print("====" * 6)
print("redis-cli -h clustercfg.redis-nonprd.orgtig.memorydb.us-west-2.amazonaws.com --tls -p 6379")


print("====" * 6)
for values in [values_dict, values_dict_prd]:
    client_id = values["client_id"]
    refresh_token = values["refresh_token"]
    expire_in = values.get('expires_in', 3000)
    print(f'HSET {client_id}:refresh_token refresh_token "{refresh_token}"')
    # print (f"EXPIRE {client_id}:refresh_token {values_dict.get('expires_in', 3000)}")
    print("")

### Optional

In [ ]:

print ("If in need to update client secret! ")
print("====" * 6)
for values in [values_dict, values_dict_prd]:
    client_secret = values["client_secret"]
    print(f'HSET {client_id}:client_secret client_secret "{client_secret}"')
    # print (f"EXPIRE {client_id}:refresh_token {values_dict.get('expires_in', 3000)}")
    print("")

In [ ]:
print("✅  Verify updated Redis cli")
print("====" * 6)
print("redis-cli -h clustercfg.redis-nonprd.orgtig.memorydb.us-west-2.amazonaws.com --tls -p 6379")
print ("AUTH nvbot-nonprod vqWn4aQpHqTzm0ar")

print("====" * 6)
for values in [values_dict, values_dict_prd]:
    client_id = values["client_id"]
    refresh_token = values["refresh_token"]
    expire_in = values.get('expires_in', 3000)
    print(f'HGET {client_id}:refresh_token refresh_token')
    # print (f"EXPIRE {client_id}:refresh_token {values_dict.get('expires_in', 3000)}")
    print("")

Then, do the same for prod Redis cli

In [ ]:
print("HSET same to prod Redis! @shiva")

### Step 4: Obtain device auth token from client token

### Complete


redis-cli -h clustercfg.redis-nonprd.orgtig.memorydb.us-west-2.amazonaws.com --tls -p 6379
clustercfg.redis-nonprd.orgtig.memorydb.us-west-2.amazonaws.com:6379> AUTH nvbot-nonprod vqWn4aQpHqTzm0ar
OK
clustercfg.redis-nonprd.orgtig.memorydb.us-west-2.amazonaws.com:6379> hget LRddW8daXcXkHW_2nvQBJt1V969xHj6m6EHf-1S4eJU:client_token
(error) ERR wrong number of arguments for 'hget' command
clustercfg.redis-nonprd.orgtig.memorydb.us-west-2.amazonaws.com:6379> hget LRddW8daXcXkHW_2nvQBJt1V969xHj6m6EHf-1S4eJU:client_token client_token
"j72xxz8GB58dz2V12a-RcJMb15QWufUBGa2JDOVYlGqY3gtMz8q11_4F_Zw03gPYqpkU591gQ93xaHFzrfynFA"
clustercfg.redis-nonprd.orgtig.memorydb.us-west-2.amazonaws.com:6379> hget LRddW8daXcXkHW_2nvQBJt1V969xHj6m6EHf-1S4eJU:sub sub
"D7ULuBfvzQvj7rOeQndJXAEbVgPmBFV0nE38cJJ32GU"